## Yelp Challenge

Dataset Documentation: <br>
https://www.yelp.com/dataset/documentation/main

In [223]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import requests
import xlearn as xl
import networkx as nx
pd.options.mode.chained_assignment = None
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
#line_count = len(open("data/review.json").readlines())
line_count = len(open("data/business.json").readlines())
business_ids, cities, states, latitudes, longitudes, stars, review_counts = [], [], [], [], [], [], []
with open("data/business.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        
businesses = pd.DataFrame(
    {"business_id": business_ids, "city": cities, "state": states, "latitude": latitudes, "longitude": longitudes, "business_average_stars": stars, "business_review_counts": review_counts }
)

100%|██████████| 192609/192609 [00:01<00:00, 99941.58it/s]


In [3]:
#line_count = len(open("data/review.json").readlines())
line_count = len(open("data/user.json").readlines())
users, review_counts, elites, average_stars, friends = [], [], [], [], []
with open("data/user.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        users += [blob["user_id"]]
        review_counts += [blob["review_count"]]
        elites += [blob["elite"]]
        average_stars += [blob["average_stars"]]
        friends += [blob["friends"]]
        
users = pd.DataFrame(
    {"user_id": users, "user_review_counts": review_counts,"elite": elites, "user_average_stars": average_stars, "friends": friends}
)

100%|██████████| 1637138/1637138 [00:18<00:00, 89544.92it/s] 


In [4]:
ratings_holdout = pd.read_csv('data/ratings_sample_holdout.csv')

In [5]:
ratings_train = pd.read_csv('data/ratings_sample_train.csv')

In [6]:
ratings_val = pd.read_csv('data/ratings_sample_cv.csv')

In [7]:
def process(df):
    df = df.drop(df.columns[0], axis =1)
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(businesses, on = 'business_id')
    return df

In [8]:
ratings_train = process(ratings_train.copy())

In [9]:
ratings_holdout = process(ratings_holdout.copy())

In [10]:
ratings_val = process(ratings_val.copy())

In [11]:
ratings_val.columns

Index(['user_id', 'business_id', 'rating', 'date', 'text', 'week_day', 'month',
       'hour', 'user_review_counts', 'elite', 'user_average_stars', 'friends',
       'city', 'state', 'latitude', 'longitude', 'business_average_stars',
       'business_review_counts'],
      dtype='object')

In [12]:
print('There are {0} rows, {1} columns in training set.'.format(ratings_train.shape[0], ratings_train.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(ratings_holdout.shape[0], ratings_holdout.shape[1]))
print('There are {0} rows, {1} columns in validation set.'.format(ratings_val.shape[0], ratings_val.shape[1]))

There are 406042 rows, 18 columns in training set.
There are 28612 rows, 18 columns in holdout set.
There are 28615 rows, 18 columns in validation set.


In [13]:
import math
def _convert_to_ffm(path, df, type, target, numerics, categories, features, encoder):
    # Flagging categorical and numerical fields
    print('convert_to_ffm - START')
    for x in numerics:
        if(x not in encoder['catdict']):
            print(f'UPDATING CATDICT: numeric field - {x}')
            encoder['catdict'][x] = 0
    for x in categories:
        if(x not in encoder['catdict']):
            print(f'UPDATING CATDICT: categorical field - {x}')
            encoder['catdict'][x] = 1

    nrows = df.shape[0]
    with open(path + str(type) + "_ffm.txt", "w") as text_file:

        # Looping over rows to convert each row to libffm format
        for n, r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow[target]))  # Set Target Variable here

            # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(encoder['catdict'].keys()):
                if(encoder['catdict'][x] == 0):
                    # Not adding numerical values that are nan
                    if math.isnan(datarow[x]) is not True:
                        datastring = datastring + " "+str(i)+":" + str(i)+":" + str(datarow[x])
                else:

                    # For a new field appearing in a training example
                    if(x not in encoder['catcodes']):
                        print(f'UPDATING CATCODES: categorical field - {x}')
                        encoder['catcodes'][x] = {}
                        encoder['currentcode'] += 1
                        print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    # For already encoded fields
                    elif(datarow[x] not in encoder['catcodes'][x]):
                        encoder['currentcode'] += 1
                        print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    code = encoder['catcodes'][x][datarow[x]]
                    datastring = datastring + " "+str(i)+":" + str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

    # print('Encoder Summary:')
    # print(json.dumps(encoder, indent=4))
    return encoder

### Create user text embeddings based on training data only

In [14]:
rating_texts = ratings_train.groupby(['user_id'])['text'].apply(lambda x: ','.join(x)).reset_index()

In [15]:
rating_texts['text'] = rating_texts['text'].str.lower()

In [16]:
rating_texts['text'] = rating_texts['text'].str.replace(r"[^a-zA-Z ]+", " ").str.strip()

In [17]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words = 'english', strip_accents = 'ascii')
vectorizer.fit(rating_texts['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents='ascii',
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [18]:
vector = vectorizer.transform(rating_texts['text'])

In [19]:
tsv = TruncatedSVD(n_components=50)

In [20]:
tsv.fit(vector)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
             random_state=None, tol=0.0)

In [21]:
transformed_tsv = tsv.transform(vector)

In [22]:
'''
wcss=[]
for i in range(77, 100):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(transformed_tsv)
    wcss.append(kmeans.inertia_)
plt.plot(range(2, 100), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
'''

"\nwcss=[]\nfor i in range(77, 100):\n    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)\n    kmeans.fit(transformed_tsv)\n    wcss.append(kmeans.inertia_)\nplt.plot(range(2, 100), wcss)\nplt.title('Elbow Method')\nplt.xlabel('Number of clusters')\nplt.ylabel('WCSS')\nplt.show()\n"

In [23]:
kmeans = KMeans(n_clusters=110, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(transformed_tsv)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=110, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [24]:
text_cluster = kmeans.predict(transformed_tsv)

In [25]:
rating_texts.loc[:,'text_cluster'] = text_cluster

In [26]:
rating_texts_features = rating_texts[['user_id','text_cluster']]

In [27]:
ratings_train= ratings_train.merge(rating_texts_features[['user_id','text_cluster']], on ='user_id', how = 'left')

In [28]:
ratings_holdout = ratings_holdout.merge(rating_texts_features[['user_id','text_cluster']], on ='user_id', how = 'left')

In [29]:
ratings_val = ratings_val.merge(rating_texts_features[['user_id','text_cluster']], on ='user_id', how = 'left')

### Create Graph Features

In [30]:
train_users = ratings_train[['user_id','friends']].drop_duplicates()

In [31]:
train_users_dict = train_users.set_index('user_id').T.to_dict('list')

In [32]:
g = nx.Graph()
g.add_nodes_from(train_users_dict.keys())

In [33]:
for k, v in train_users_dict.items():
    for i in v[0].split(','):
        g.add_edge(k,i.strip())    

In [34]:
sub_graphs = nx.connected_component_subgraphs(g)

In [35]:
sgs =[]
for i, sg in enumerate(sub_graphs):
    sgs += [sg]

In [36]:
fin_sgs = []
for i in sgs:
    if len(i.nodes()) >=5:
        fin_sgs +=[i]

In [37]:
graph_user_ids, graph_ids = [],[]
num_id = 0
for graph in fin_sgs:
    for node in graph.nodes():
        graph_user_ids += [node]
        graph_ids += [num_id]
    num_id += 1

In [38]:
social_graphs = pd.DataFrame(
    {"user_id": graph_user_ids, "graph_cluster": graph_ids}
)

In [39]:
ratings_train= ratings_train.merge(social_graphs, on ='user_id', how = 'left')

In [40]:
ratings_holdout = ratings_holdout.merge(social_graphs, on ='user_id', how = 'left')

In [41]:
ratings_val = ratings_val.merge(social_graphs, on ='user_id', how = 'left')

### Create Location Features

In [42]:
X = ratings_train[['longitude','latitude']]

In [43]:
''' 
EXPLORATORY ANALYSIS TO DETERMINE NUMBER OF CLUSTERS. DON'T RUN
HERE USING THE EBLOW METHOD WE CHOOSE NCLUSTERs OF 10 

X = ratings_train[['longitude','latitude']]
wcss = []
for i in range(2, 100):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(2, 50), wcss[0:48])
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()
'''

" \nEXPLORATORY ANALYSIS TO DETERMINE NUMBER OF CLUSTERS. DON'T RUN\nHERE USING THE EBLOW METHOD WE CHOOSE NCLUSTERs OF 10 \n\nX = ratings_train[['longitude','latitude']]\nwcss = []\nfor i in range(2, 100):\n    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)\n    kmeans.fit(X)\n    wcss.append(kmeans.inertia_)\nplt.plot(range(2, 50), wcss[0:48])\nplt.title('Elbow Method')\nplt.xlabel('Number of clusters')\nplt.ylabel('WCSS')\nplt.show()\n"

In [44]:
kmeans = KMeans(n_clusters=10, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [45]:
ratings_train.loc[:,'location_cluster'] = kmeans.predict(ratings_train[['longitude','latitude']])
ratings_holdout.loc[:,'location_cluster'] = kmeans.predict(ratings_holdout[['longitude','latitude']])
ratings_val.loc[:,'location_cluster'] = kmeans.predict(ratings_val[['longitude','latitude']])

In [46]:
ratings_train.text_cluster.fillna('999', inplace=True)
ratings_holdout.text_cluster.fillna('999', inplace=True)
ratings_val.text_cluster.fillna('999', inplace=True)

In [47]:
ratings_train.location_cluster.fillna('999', inplace=True)
ratings_holdout.location_cluster.fillna('999', inplace=True)
ratings_val.location_cluster.fillna('999', inplace=True)

In [48]:
ratings_train.graph_cluster.fillna('999', inplace=True)
ratings_holdout.graph_cluster.fillna('999', inplace=True)
ratings_val.graph_cluster.fillna('999', inplace=True)

### FFM CLASS

In [219]:
class FFMModel:

    def __init__(self, train, test, config, suffix = None):
        self.train_df = train
        self.test_df = test
        self.model = xl.create_ffm()
        self.suffix = suffix
        self.config = config
        self.preds = None 
          
    def __configure(self):
        destination = self.config['destination']
        label = self.config['label']
        numerical_columns  = self.config['numerical_columns']
        categorical_columns  = self.config['categorical_columns']
        all_columns  = numerical_columns + categorical_columns
              
        encoder = {"currentcode": len(self.config['numerical_columns']),
           "catdict": {},
           "catcodes": {}}
        
        encoder = _convert_to_ffm(destination, self.train_df , 'train', label, numerical_columns, categorical_columns, all_columns, encoder)
        encoder = _convert_to_ffm(destination, self.test_df , 'test', label, numerical_columns, categorical_columns, all_columns, encoder)
        
    def train(self, params = None):
        encoder = self.__configure()
        self.model.setTrain(self.config['destination']+'train_ffm.txt')
        self.model.setValidate(self.config['destination']+'test_ffm.txt')
        self.model.setTest(self.config['destination']+'test_ffm.txt')
        
        if not params:
            params = {'task': 'reg',
                     'lr': 0.2,
                     'lambda': 0.002,
                     'metric': 'auc'}
        
        self.model.fit(params, self.config['model_destination']+self.config['model_name']+'.out')

    def evaluate_on_val(self):
        self.model.predict(self.config['model_destination'] + self.config['model_name']+'.out', self.config['output_destination']+'predictions.txt')
        preds = pd.read_csv(self.config['output_destination']+'predictions.txt', sep=" ", header=None)
        preds.columns = ['prediction']
        preds.prediction = np.clip(preds.prediction,0.0,5.0)
        self.preds = preds
        return preds            
    
    def get_regression_metrics(self):
        if self.preds is None:
            self.evaluate_on_val()
        
        predictions = self.evaluate_on_val()
        test_df = self.test_df.copy()
        test_df['preds']  = np.clip(predictions.values,0.0,5.0)
        
        r2 = r2_score(test_df['rating'], test_df['preds'])
        mse = mean_squared_error(test_df['rating'], test_df['preds'])
        mae = mean_absolute_error(test_df['rating'], test_df['preds'])
        
        print('R2: ', r2, ' MSE: ',mse, ' MAE: ', mae)
        return r2,mse,mae
            

### Metrics with Just userid and business id

In [232]:
baseline_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'baseline',
    'output_destination': 'output/'
}

In [233]:
baseline_FFM = FFMModel(ratings_train, ratings_val, baseline_config)

In [234]:
%%capture
baseline_FFM.train()

In [235]:
r2, mse, mae = baseline_FFM.get_regression_metrics()

R2:  0.16359044241141085  MSE:  1.8428614957708407  MAE:  1.115170516826839


##### Metrics with Baseline + Business Location

In [85]:
user_business_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'user_business',
    'output_destination': 'output/'
}

In [86]:
user_business_FFM = FFMModel(ratings_train, ratings_val, user_business_config)

In [87]:
%%capture
user_business_FFM.train()

In [88]:
r2, mse, mae = user_business_FFM.get_regression_metrics()

R2:  0.16561123600483585  MSE:  1.8384090804792699  MAE:  1.096584375956666


##### Metrics with Baseline + Location Cluster

In [89]:
location_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','location_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'location',
    'output_destination': 'output/'
}

In [90]:
location_FFM = FFMModel(ratings_train, ratings_val, location_config)

In [91]:
%%capture
location_FFM.train()

In [92]:
r2, mse, mae = location_FFM.get_regression_metrics()

R2:  0.16952274567049086  MSE:  1.8297908497479602  MAE:  1.0989993160580116


##### Metrics with User and Business Information + Text Cluster

In [93]:
text_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','text_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'text',
    'output_destination': 'output/'
}
text_FFM = FFMModel(ratings_train, ratings_val, text_config)

In [94]:
%%capture
text_FFM.train()

In [95]:
r2, mse, mae = text_FFM.get_regression_metrics()

R2:  0.15673734312059995  MSE:  1.8579609320400061  MAE:  1.1026677958063953


##### Metrics with User and Business Information + Graph Cluster

In [96]:
graph_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','graph_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'graph',
    'output_destination': 'output/'
}
graph_FFM = FFMModel(ratings_train, ratings_val, graph_config)

In [97]:
%%capture
graph_FFM.train()

In [98]:
r2, mse, mae = graph_FFM.get_regression_metrics()

R2:  0.1665249930806647  MSE:  1.836395799167196  MAE:  1.0960562833129477


##### Metrics All clusters

In [99]:
all_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','graph_cluster','location_cluster','text_cluster'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'all_clusters',
    'output_destination': 'output/'
}
all_FFM = FFMModel(ratings_train, ratings_val, all_config)

In [100]:
%%capture
all_FFM.train()

In [101]:
r2, mse, mae = all_FFM.get_regression_metrics()

R2:  0.15706300571220033  MSE:  1.857243399528298  MAE:  1.0948153850008737


### Run Time Analysis

FFM runs in linear time which leads to fast runtime. Our model takes just two minutes to run

In [364]:
%%capture
import time
start_time = time.time()
location_FFM = FFMModel(ratings_train, ratings_val, location_config)
location_FFM.train()

In [365]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 80.56991696357727 seconds ---


In [366]:
%%capture
start_time = time.time()
location_FFM.get_regression_metrics()

In [367]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.16264057159423828 seconds ---


### Serendipity and Ranking Metrics

In [ ]:
ratings_entire_df = ratings_train.append(rating_val).append(ratings_holdout)

In [314]:
unique_city_businesses = ratings_entire_df[['city','business_id']].drop_duplicates()
unique_cities = unique_city_businesses.groupby('city').count()['business_id']
unique_cities = unique_cities[unique_cities > 100]
out = pd.DataFrame()
for city in unique_cities.index:
    tmp = ratings_val[(ratings_val['city'] ==city) &
                              (ratings_val['rating'] >ratings_val['user_average_stars'])]
    if len(tmp['user_id'].unique())>4:
        
        ###this weird sampling technique is to ensure we dont' sample the same user twice in a same city
        five_users = np.random.choice(tmp['user_id'].unique(),5, replace = False)
        row = tmp[tmp['user_id'].isin(five_users)].groupby('user_id', group_keys=False).apply(lambda df: df.sample(1))
        out = out.append(row)

In [327]:
predict_df = out[['user_id','city','state']]

In [328]:
predict_df = predict_df.merge(unique_city_businesses, on = 'city')

In [330]:
predict_df = predict_df.merge(businesses[['business_id','latitude','longitude']], on ='business_id')

In [332]:
predict_df.loc[:,'location_cluster'] = kmeans.predict(predict_df[['longitude','latitude']])

In [337]:
predict_df['rating'] = 2.5 

In [338]:
location_config = {   'destination': 'data/',
    'categorical_columns':['business_id','user_id','location_cluster','city','state'],
    'numerical_columns' : [],
    'label' : 'rating',
    'model_destination' : 'trained_models/',
    'model_name' : 'location',
    'output_destination': 'output/'
}

In [339]:
location_FFM = FFMModel(ratings_train, predict_df, location_config)

In [340]:
%%capture
location_FFM.train()

In [341]:
predictions = location_FFM.evaluate_on_val()

In [342]:
predict_df['predictions'] = predictions['prediction']

In [343]:
top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()

In [344]:
all(top_10_recs.groupby('city')['user_id'].count()==50)

True

In [347]:
cnt =0
serendipity = 0
for row in out.iterrows():
    row_values = row[1]
    top_10 = predict_df.loc[top_10_recs[top_10_recs['user_id'] == row_values['user_id']].level_2]['business_id']
    ###In top 10
    if row_values['business_id'] in top_10.values:
        cnt+=1
    user_history = ratings_train[ratings_train['user_id'] == row_values['user_id']]    
    been_there = [i for i in top_10.values if i in  user_history.business_id.values]
    serendipity += 1-len(been_there)/10

### 15% of users had their latest restaurant in our top 10 recommendations

In [348]:
cnt/len(out)

0.15294117647058825

### 96.5% of our recommendations were restaurant that user hasn't been

In [349]:
serendipity/len(out)

0.9652941176470581

In [350]:
predict_df = predict_df.reset_index()

In [351]:
predict_df.columns

Index(['index', 'user_id', 'city', 'state', 'business_id', 'latitude',
       'longitude', 'location_cluster', 'rating', 'predictions'],
      dtype='object')

In [352]:
top_10_recs.head(1)

,user_id,city,level_2,predictions
0,-0ZildkQkxhgqGWpkzv9Jg,Brampton,4990,4.04152


In [353]:
predict_df.head(1)

,index,user_id,city,state,business_id,latitude,longitude,location_cluster,rating,predictions
0,0,2pRiLoNo3H4hzNhXWE9f4w,Akron,OH,lMyCs9TssL9fbQR_Xiekcw,41.118293,-81.588272,8,2.5,4.14656


In [354]:
analysis_df = predict_df.merge(top_10_recs, left_on = ['user_id','city','index'], right_on = ['user_id','city','level_2'])

In [355]:
all(analysis_df.groupby('city')['business_id'].count() ==50)

True

### 25% of our recommendations were unique business ids in that city(Coverage)

In [356]:
(analysis_df.groupby('city')['business_id'].nunique()/50).values.mean()

0.25441176470588234

In [357]:
predict_df['rankings']=predict_df.groupby(['city','user_id'])['predictions'].rank("first",ascending = False)

In [358]:
running_rankings =0
for row in out.iterrows():
    row_values = row[1]
    user_recs = predict_df[(predict_df['user_id']==row_values['user_id'])
                        &(predict_df['city']==row_values['city'])
                         & (predict_df['business_id']==row_values['business_id'])
                          ]
    assert len(user_recs)==1
    running_rankings += user_recs['rankings'].sum()

### Average ranking of the latest restaurant that the user visited 

In [359]:
running_rankings / len(out)

493.7470588235294